# Recognizing Handwritten Digits using Scikit-Learn

MNIST dataset: https://en.wikipedia.org/wiki/MNIST_database

* 28x28 (784 pixels total) images of handwritten digits
* each image has a label 0-9
* ~~70,000~~ 42,000 total images & labels
* our goal is to correctly guess the label when we can only see pixel data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
import pandas as pd

%matplotlib inline 

mnist = pd.read_csv('../MNIST/train.csv') # only using train, because test has no labels

print(mnist.shape)

(42000, 785)


# Machine Learning Rundown

"Machine Learning is the science (and art) of programming computers so they can *learn from data.*"

* Aurélien Géron, *Hands-On Machine Learning with Scikit-Learn & Tensorflow*

**Some useful resources**

* Scikit-Learn's introduction to machine learning: http://scikit-learn.org/stable/tutorial/basic/tutorial.html
* MIT OpenCourseWare: https://www.youtube.com/watch?v=h0e2HAPTGF4
* kaggle - datasets and ML competitions: https://www.kaggle.com/datasets

**Example: Spam-filter**

* Learns to flag spam from given examples of spam emails and regular, "ham", emails
* This set of example emails is called the *training set*
* Each email in the training set is called an *instance*
* Each instance contains *data* (contents of the email) and a *target* ('spam' or 'ham')
* Each point of data within an instance is a *feature*
* For training purposes, we separate the data and target

**Math notation**

* Uppercase Bold = matrices
* lowercase bold = vectors
* Italics = scalars
* Rows are indexed by subscript, Columns by superscript

$\mathbf{X}$ is a matrix containing the data portion of our dataset (no labels)

* $\mathbf{X}$ has $n$ rows (instances) and $m$ columns (features)

The i-th row of the matrix, $\mathbf{x}_i$, is one instance in our dataset.

The j-th element of the i-th row is the value of one feature for a given instance, denoted $x_i^j$

$\mathbf{y}$ is the target vector containing a label for each instance 

The i-th element of the target vector is a label for a single instance, $y_i$

* Sometimes our data doesn't have labels. When we have labels for training, it's called *Supervised Learning*. When we don't, it's *Unsupervised Learning.* We'll stick to Supervised Learning for now.

Our model is a function, $h$, which takes in an instance, $\mathbf{x}_i$, and gives us a *Predicted Label*, $\hat{y_i}$

$$\large{ h(\mathbf{x}_i) = \hat{y_i} }$$

if $\hat{y_i} = y_i$, then our model guessed the label perfectly for the i-th instance

We can also run our model on all instanes in our dataset in one go:

$$\large{ h(\mathbf{X}) = \mathbf{\hat{y}} }$$

In practice, we split the given dataset into a *Training Set* and *Test Set*, so we can evaluate our models after we train them.
*It is very important that we do not let our models see the test data during training!* Our goal is not to memorize our data, but to create a model capable of classifyig data it has never seen.

# Separate data and target labels

* Pandas dataframe/series documentation: https://pandas.pydata.org/pandas-docs/stable/dsintro.html
* iloc takes in positional index (ints) and returns a dataframe or series

In [10]:
data = mnist.iloc[:,1:] # returns a numpy array
target = mnist.iloc[:,0]
# 784 features (pixels), 42,000 instances (images)
print("data shape: ", data.shape)
print("target shape: ", target.shape)
print(type(data))
print(type(target))

data shape:  (42000, 784)
target shape:  (42000,)
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


# Test/Train Split

In [7]:
X = data.values # converts to numpy array
y = target.values
print(type(X), " ", type(y))
# 37,000 for training, 5,000 for testing
X_train, X_test = X[:37000], X[37000:]
y_train, y_test = y[:37000], y[37000:]

<class 'numpy.ndarray'>   <class 'numpy.ndarray'>


# Display A Random Training Image

In [ ]:
rand_index = np.random.randint(0, X_train.shape[0])
print('Showing image ', rand_index)
plt.imshow(X_train[rand_index].reshape(28,28), cmap='Greys')
plt.axis('off')
print('Label: ', target[rand_index])

# Binary Classifier

Easy starting point. Output is True or False (in the class, or not in the class)

In this case, we'll recognize whether a digit is 5.

In [ ]:
# First, we need to restructure our labels
y_train_5 = (y_train == 5) 
y_test_5 = (y_test == 5)
# now, instead of digits 0-9, we have 0's for 'not 5' and 1's for '5' - Perfect!

# Scikit-Learn

* list of classification models: http://scikit-learn.org/stable/supervised_learning.html
* To start, we will use a SGDClassifier http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html
    - SGD is the name of the training algorithm, "Stochastic Gradient Descent." We'll learn much more about Gradient Descent algorithms soon!
    - This is a linear model, meaning it finds linear boundaries between classes (remember, our classes to start out with are 'five' and 'not five')
    - The main reason I've selected this model is that it's simplicity allows for much faster training on our high-dimensional dataset. The trade-off is that it will be less accurate than more complex models.
* To train models in sklearn, we only have to call the fit() function and pass in our data and labels

# Train

In [ ]:
from sklearn.linear_model import SGDClassifier

# there are many parameters we can set here, but they all have default values, so we can safely leave this blank
classifier = SGDClassifier() 
classifier.fit(X_train, y_train_5) # This line alone trains the model. 
# We'll go over how to implement our own fit() function soon!

# ignore the warning - the default parameters for this class work fine for our purposes

# Make a Prediction

In [ ]:
rand_index = np.random.randint(0, 5000)
# once the model has been trained, we only have to call the predict() function!
prediction = classifier.predict([X_test[rand_index]])
print("Sample ", rand_index, ":")
print("Predicted Label: ", prediction)
print("Actual Label (binary): ", y_test_5[rand_index])
print("Actual Label (multi-class): ", y_test[rand_index])
plt.imshow(X_test[rand_index].reshape(28,28), cmap='Greys')
plt.axis('off')
plt.show()

Wow, it worked!! Or maybe it didn't. Either way, one example from the test set doesn't tell us much, and we isolated 5,000 instances for a reason. Fortunately, we can run predict() on the entire test set and get back an array of 5,000 predicted labels. Using that, we can really evaluate how effective our model is.

# Model Evaluation

In [ ]:
from sklearn.metrics import accuracy_score

y_test_5_pred = classifier.predict(X_test)
score = accuracy_score(y_test_5, y_test_5_pred)
print(score)

Wow, that's a great score! Guess our simple model did pretty well on it's own? Not quite.

Just because we are doing a binary classification task doesn't mean that our data is split evenly into two classes! In this case, only 10% of the samples are in the 'five' class, and the rest are 'not five', *so a model that spits out [ False ] Every time would have 90% accuracy!*

# Confusion Matrix

We can't rely on a single metric to evaluate our model. One useful metric is the *Confusion Matrix.* A confusion matrix shows us how many instances are classified correctly and incorrectly for both of our classes. More precisely,

$$\text{confusion matrix} = \begin{bmatrix} \text{True Negative} & \text{False Positive} \\ \text{False Negative} & \text{True Positive} \end{bmatrix}$$

In [ ]:
from sklearn.metrics import confusion_matrix
conf_mx = confusion_matrix(y_test_5, y_test_5_pred)
print(conf_mx)

# Precision and Recall

Two more useful values we can extract from the confusion matrix are *Precision* and *Recall.*

https://en.wikipedia.org/wiki/Precision_and_recall

* Precision is the proportion of instances that were correctly labeled as 'True' out of all the 'True' predictions
    - How many of our selected items were classified correctly?
    - Higher precision => fewer false positives
* Recall is the proportion of instances that were correctly labeled as 'True' out of all the instances that actually are 'True'
    - How many of the total relevant items did we get?
    - Higher recall => fewer false negatives
    
More formally,

$\text{Precision} = \large{\frac{\text{True Positive}}{\text{True Positive + False Positive}}}$

$\text{Recall} = \large{\frac{\text{True Positive}}{\text{True Positive + False Negative}}}$

In [ ]:
from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_test_5, y_test_5_pred)
recall = recall_score(y_test_5, y_test_5_pred)
print("Precision: ", precision)
print("Recall: ", recall) 

# Multiclass Classification

